In [3]:
import pandas as pd

stock_name = "AAPL"
file_path = f"data/production/{stock_name}_merged_data.csv"

df = pd.read_csv(file_path)
data_dict = df.set_index("date")["Close"].astype(str).to_dict()

In [1]:
# from flask import Flask, render_template, request, jsonify
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pandas as pd

# app = Flask(__name__)

# Global variable for look_backP
look_back = 60


# Function to download and preprocess stock data
def get_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data


# Function to create dataset with lookback
def create_dataset(data, look_back=1):
    X, Y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data[i : (i + look_back), 0])
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)


# Function to train LSTM model
def train_lstm_model(X, Y, epochs=100, batch_size=32):
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=(1, look_back)))
    model.add(Dropout(0.5))
    model.add(LSTM(units=100, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(units=100))
    model.add(Dropout(0.5))
    model.add(Dense(units=1))

    optimizer = Adam(lr=0.0001)

    model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["accuracy"])
    history = model.fit(
        X, Y, epochs=epochs, batch_size=batch_size, validation_split=0.2
    )  # Added validation_split
    return model, history

2024-02-16 07:49:31.388915: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 07:49:31.458691: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 07:49:31.458737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 07:49:31.460116: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 07:49:31.468614: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 07:49:31.469649: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
data = {"symbol": "AAPL", "start-date": "2022-02-16", "end-date": "2024-02-16"}
print("dataf", data)
symbol = data["symbol"]
start_date = data["start-date"]
end_date = data["end-date"]
stock_data = get_stock_data(symbol, start_date, end_date)
if stock_data.empty:
    print("No data found for the period")
    # return jsonify({'error': 'No data found for the specified time period.'}), 400

prices = stock_data["Close"].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
prices_normalized = scaler.fit_transform(prices)
X, Y = create_dataset(prices_normalized, look_back)  # Reshape X here
X = np.reshape(X, (X.shape[0], 1, look_back))  # Reshape X to match expected shape

model, history = train_lstm_model(X, Y)

dataf {'symbol': 'AAPL', 'start-date': '2022-02-16', 'end-date': '2024-02-16'}


/home/codespace/.python/current/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


Epoch 1/100
11/11 [==============================] - 5s 117ms/step - loss: 0.2162 - accuracy: 0.0028 - val_loss: 0.2873 - val_accuracy: 0.0000e+00
Epoch 2/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0624 - accuracy: 0.0028 - val_loss: 0.0200 - val_accuracy: 0.0112
Epoch 3/100
11/11 [==============================] - 0s 5ms/step - loss: 0.0420 - accuracy: 0.0028 - val_loss: 0.0829 - val_accuracy: 0.0112
Epoch 4/100
11/11 [==============================] - 0s 5ms/step - loss: 0.0310 - accuracy: 0.0028 - val_loss: 0.0107 - val_accuracy: 0.0112
Epoch 5/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0350 - accuracy: 0.0028 - val_loss: 0.0240 - val_accuracy: 0.0112
Epoch 6/100
11/11 [==============================] - 0s 9ms/step - loss: 0.0243 - accuracy: 0.0028 - val_loss: 0.0087 - val_accuracy: 0.0112
Epoch 7/100
11/11 [==============================] - 0s 10ms/step - loss: 0.0218 - accuracy: 0.0028 - val_loss: 0.0125 - val_accuracy: 0.0112
Epoch 

In [3]:
last_price = prices_normalized[-look_back:]

array([[0.90887942],
       [0.89779723],
       [0.90696398],
       [0.88863047],
       [0.88616764],
       [0.89451353],
       [0.88042133],
       [0.88835678],
       [0.90600637],
       [0.8812422 ],
       [0.93583251],
       [0.92078269],
       [0.94746209],
       [0.96716386],
       [0.93254881],
       [0.95348209],
       [0.99794782],
       [1.        ],
       [0.99261194],
       [0.96962647],
       [0.98399236],
       [0.95512384],
       [0.95307145],
       [0.93829533],
       [0.93077032],
       [0.93213837],
       [0.93802164],
       [0.92365575],
       [0.82938842],
       [0.81037078],
       [0.77835551],
       [0.76836767],
       [0.82829385],
       [0.82254754],
       [0.83691343],
       [0.82870429],
       [0.83321929],
       [0.80188815],
       [0.78889031],
       [0.87029696],
       [0.91038442],
       [0.94226295],
       [0.95991233],
       [0.95060883],
       [0.94609383],
       [0.92215074],
       [0.9127103 ],
       [0.862

In [ ]:
last_price = prices_normalized[-look_back:]
last_price = np.reshape(last_price, (1, 1, look_back))
predicted_price_normalized = model.predict(last_price)
predicted_price = scaler.inverse_transform(predicted_price_normalized)

# Prepare previous and predicted data for visualization
prev_dates = stock_data.index[-look_back:].tolist()
prev_prices = (
    scaler.inverse_transform(prices_normalized[-look_back:]).flatten().tolist()
)
predicted_date = stock_data.index[-1]
predicted_price = float(predicted_price[0][0])

getting data from jan 2020

In [12]:
api_key = "0VQBNxZe7WZ19GVMOiqTVKUN0oipj_HT"  # Replace with your actual API key

Code to get the data of news about 10 articles , for previous data since jan 2020 . If the current data is not present it will fetch the data automatically . Use this to visualise the json file https://jsonviewer.stack.hu/. Do not toch the stock data file 

In [28]:
import json
import os
import requests
from datetime import datetime, timedelta

# Global variable for the starting point
START_DATE = "2024-02-01"

def fetch_data_for_date(ticker, date, api_key):
    """
    Fetches stock data from an API for a single date.
    """
    url = f"https://api.polygon.io/v2/reference/news?ticker={ticker}&apiKey={api_key}&date={date}&limit=50"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json().get('results', [])
        if data:
            return {date: data}
    else:
        print(f"Failed to fetch data for {ticker} on {date}: HTTP {response.status_code}")
    return {}

def update_stock_data(ticker, new_data):
    """
    Updates or creates a JSON file for the ticker with new data.
    """
    file_path = os.path.join('stock_data', f'{ticker}.json')
    with open(file_path, 'w') as file:
        json.dump(new_data, file, indent=4)

def check_and_update_data(ticker, api_key):
    """
    Checks if the current date's data is present and updates if necessary by fetching data day by day.
    """
    current_date = datetime.now().strftime('%Y-%m-%d')
    file_path = os.path.join('stock_data', f'{ticker}.json')
    
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            data = json.load(file)
            last_date = max(data.keys())
            start_date = datetime.strptime(last_date, '%Y-%m-%d') + timedelta(days=1)
    else:
        print(f"No data file found for ticker {ticker}. Starting initial fetch...")
        data = {}
        start_date = datetime.strptime(START_DATE, '%Y-%m-%d')

    # Loop through each date from start_date to current_date
    date_to_fetch = start_date
    while date_to_fetch.strftime('%Y-%m-%d') <= current_date:
        date_str = date_to_fetch.strftime('%Y-%m-%d')
        if date_str not in data:
            print(f"Fetching data for {date_str}...")
            new_data = fetch_data_for_date(ticker, date_str, api_key)
            data.update(new_data)
        date_to_fetch += timedelta(days=1)
    
    # Update the stock data file with all fetched data
    if data:
        update_stock_data(ticker, data)
        print("Data has been updated with new information.")
    else:
        print("No new data available to update.")

# Example usage
ticker = 'AMZN'  # Replace with the ticker you're interested in
check_and_update_data(ticker, api_key)

def get_stock_data_polygon(ticker)
    check_and_update_data(ticker, api_key)
    file_path = os.path.join('stock_data', f'{ticker}.json')
    with open(file_path, 'w') as file:
        json_data=json.load(file)
    print(f"json_data_type:{type(json_data)}\n{json_data}")
    return json_data

    

No data file found for ticker AMZN. Starting initial fetch...
Fetching data for 2024-02-01...
Fetching data for 2024-02-02...
Fetching data for 2024-02-03...
Fetching data for 2024-02-04...
Fetching data for 2024-02-05...
Fetching data for 2024-02-06...
Fetching data for 2024-02-07...
Fetching data for 2024-02-08...
Fetching data for 2024-02-09...
Fetching data for 2024-02-10...
Fetching data for 2024-02-11...
Fetching data for 2024-02-12...
Fetching data for 2024-02-13...
Fetching data for 2024-02-14...
Fetching data for 2024-02-15...
Fetching data for 2024-02-16...
Fetching data for 2024-02-17...
Fetching data for 2024-02-18...
Data has been updated with new information.


In [4]:
import os, json


def get_ticker_headlines(ticker):
    """



    Returns a structured list of dates and headlines for the given ticker symbol.
    """


    # check the data status if not available send update the json database .

    check_and_update_data(ticker, api_key)

    file_path = os.path.join("stock_data", f"{ticker}.json")

    response_structure = []


    if os.path.exists(file_path):

        with open(file_path, "r") as file:
            data = json.load(file)
            for date, articles in data.items():
                headlines = [
                    article["title"] for article in articles
                ]  # Assuming 'title' holds the headline
                response_structure.append({"date": date, "headlines": headlines})

    else:

        print(f"No data file found for ticker {ticker}.")

        return []

    return response_structure



# Example usage



ticker = "AAPL"  # Replace with the ticker you're interested in



headlines_data = get_ticker_headlines(ticker)



print(json.dumps(headlines_data, indent=4))

[
    {
        "date": "2024-02-01",
        "headlines": [
            "The Smartest Dividend Stocks to Buy With $400 Right Now",
            "Mark Zuckerberg Offers Why Tech Layoffs Won't End Anytime Soon: 'It Was Really Tough'",
            "Think the S&P 500 is top-heavy? Five stocks command 60% of this market.",
            "\u2018I felt humiliated\u2019: She slipped the waiter her credit card on her way to the restroom. Is it emasculating for a woman to pay for dinner on a first date?",
            "Will Coinbase Be a Trillion-Dollar Stock by 2035?",
            "3 Unstoppable Dividend Stocks That Could Pay You for Life",
            "Bullish bets on Nvidia, Magnificent Seven near most crowded levels of past year",
            "Why these four \u2018Magnificent Seven\u2019 stocks are earnings-season winners",
            "Warren Buffett Sold Nearly $2 Billion Worth of Apple Last Quarter. Here's Why.",
            "Want your stock picks to beat index funds? Look at companies with 

In [ ]:
{
    "date": "date",
    "headlines": [
        "headline 1",
        "headline 2",
    ],
    "date": "date",
    "headlines": ["headline 1", "headline 2"],
}